In [38]:
import os
import re
import random

In [39]:
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.matcher import Matcher

In [40]:
spacy.prefer_gpu()

nlp = spacy.load("pt_core_news_lg")

In [41]:
#arquivo_sem_erro = '../Base/data.v2/amostra_base_sem erros.txt'
#v3 = '../Base/data.v3/amostra_base_v3.txt'
arquivo_sem_erro = './tres_ultimas_linhas.txt'
v3 = './com_erros.txt'

In [42]:
conteudo = open(arquivo_sem_erro, 'r', encoding='utf-8').readlines()
total_linhas = int(len(conteudo)*0.5)
indices_selecionados = random.sample(range(0, len(conteudo)-1), total_linhas)
print (len(conteudo), total_linhas, len(indices_selecionados), conteudo[0:5])

16 8 8 ['O cachorro é doméstico.\n', 'O carro bonito.\n', 'O professor está doente.\n', 'As frutas são saudáveis.\n', 'O homem é alto.\n']


In [43]:
def posses(doc):
    posses_list = []
    for token in doc:
        if str(token.pos_) == "ADJ" or str(token.pos_) == "NOUN" or str(token.pos_) == "DET" or str(token.pos_) == "PRON":
            if str(token.pos_) not in posses_list:
                posses_list.append(str(token.pos_))
    return posses_list

In [44]:
def changeGender(token):
    try:
        if str(token.morph.get('Gender')[0]) == 'Masc':
            if token.text.endswith('o'):
                return token.text[:-1] + 'a'
            elif token.text.endswith('or'):
                return token.text[:-2] + 'ora'
            elif token.text.endswith('ão'):
                return token.text[:-2] + 'ã'
            elif token.text.endswith('ês'):
                return token.text[:-2] + 'esa'
            elif token.text.endswith('r'):
                return token.text + 'a'
            elif token.text.endswith('l'):
                return token.text + 'a'
            elif token.text.endswith('z'):
                return token.text + 'a'
            elif token.text.endswith('e'):
                return token.text + 'a'
        elif str(token.morph.get('Gender')[0]) == 'Fem':
            if token.text.endswith('a'):
                return token.text[:-1] + 'o'
            elif token.text.endswith('ora'):
                return token.text[:-1]
            elif token.text.endswith('ã'):
                return token.text + 'o'
            elif token.text.endswith('esa'):
                return token.text[:-1] + 'o'
            elif token.text.endswith('r'):
                return token.text[:-1] + 'or'
            elif token.text.endswith('l'):
                return token.text[:-1] + 'or'
            elif token.text.endswith('z'):
                return token.text[:-1] + 'or'
            elif token.text.endswith('e'):
                return token.text[:-1] + 'or'
    except:
        print('Erro: ', token.text)
    return token.text

In [45]:
def changeNumber(token):
    try:
        if str(token.morph.get('Number')[0]) == 'Sing':
            if token.text.endswith('s'):
                return token.text[:-1]
            elif token.text.endswith('ão'):
                return token.text[:-2] + 'ões'
            elif token.text.endswith('ês'):
                return token.text[:-2] + 'eses'
            elif token.text.endswith('l'):
                return token.text + 's'
            elif token.text.endswith('r'):
                return token.text + 'es'
            elif token.text.endswith('z'):
                return token.text + 'es'
            elif token.text.endswith('m'):
                return token.text[:-1] + 'ns'
            else:
                return token.text + 's'
        elif str(token.morph.get('Number')[0]) == 'Plur':
            if token.text.endswith('s'):
                return token.text + 'es'
            elif token.text.endswith('ão'):
                return token.text[:-2] + 'ões'
            elif token.text.endswith('ês'):
                return token.text[:-2] + 'eses'
            elif token.text.endswith('l'):
                return token.text + 'es'
            elif token.text.endswith('r'):
                return token.text + 'es'
            elif token.text.endswith('z'):
                return token.text + 'es'
            elif token.text.endswith('m'):
                return token.text[:-1] + 'ns'
            else:
                return token.text + 's'
    except:
        print('Erro: ', token.text)
    return token.text

In [46]:
def changeWord(token):
    which = random.randint(0, 1)
    if str(token.pos_) == "NOUN" or str(token.pos_) == "PRON" or str(token.pos_) == "DET" or str(token.pos_) == "ADJ":
        if which == 0:
            return changeGender(token)
        else:
            return changeNumber(token)
    return token.text

In [47]:
def pick_and_remove(lst):
    index = random.randint(0, len(lst) - 1)
    picked_string = lst[index]
    lst.remove(picked_string)
    return picked_string

In [48]:
def insert_error(text):
    doc = nlp(text)
    posses_list = posses(doc)
    num_erros = random.randint(1, len(posses_list))
    words = []
    for i in range(num_erros):
        pos = pick_and_remove(posses_list)
        for token in doc:
            word = token.text
            if str(token.pos_) == pos:
                word = changeWord(token)
            if len(words) <= token.i:
                words.append(word)
            else:
                words[token.i] = word
    return Doc(doc.vocab, words=words)

In [49]:
novo_conteudo = []
msg = 'Iniciando inserção de erros...'
print(msg)
msg = 'Total de linhas: ' + str(len(conteudo))
print(msg)
erros = 0
normais = 0
for index, linha in enumerate(conteudo):
    if index not in indices_selecionados:
        novo_conteudo.append(linha)
        normais += 1
    else:
        novo_conteudo.append(insert_error(linha).text)
        print("oi")
        erros += 1
    print(f'{erros} já foram inseridos e {normais} linhas normais foram copiadas. faltam {len(conteudo) - (erros + normais)} linhas.')

Iniciando inserção de erros...
Total de linhas: 16


oi
1 já foram inseridos e 0 linhas normais foram copiadas. faltam 15 linhas.
1 já foram inseridos e 1 linhas normais foram copiadas. faltam 14 linhas.
oi
2 já foram inseridos e 1 linhas normais foram copiadas. faltam 13 linhas.
2 já foram inseridos e 2 linhas normais foram copiadas. faltam 12 linhas.
2 já foram inseridos e 3 linhas normais foram copiadas. faltam 11 linhas.
2 já foram inseridos e 4 linhas normais foram copiadas. faltam 10 linhas.
oi
3 já foram inseridos e 4 linhas normais foram copiadas. faltam 9 linhas.
3 já foram inseridos e 5 linhas normais foram copiadas. faltam 8 linhas.
oi
4 já foram inseridos e 5 linhas normais foram copiadas. faltam 7 linhas.
oi
5 já foram inseridos e 5 linhas normais foram copiadas. faltam 6 linhas.
oi
6 já foram inseridos e 5 linhas normais foram copiadas. faltam 5 linhas.
oi
7 já foram inseridos e 5 linhas normais foram copiadas. faltam 4 linhas.
7 já foram inseridos e 6 linhas normais foram copiadas. faltam 3 linhas.
7 já foram inseridos e 7

In [50]:
with open(v3, 'w', encoding='utf-8') as f:
    f.writelines(novo_conteudo)